This notebook will be used to visualise the best strategy for winning the game.

In [26]:
def check_won(field, amphipods):
    if all(amp.is_endgame(field) for amp in amphipods):
        return True
    return False

In [3]:
def check_neighbours(space, field, neighbour_set):
    try:
        current_value = field[space[0], space[1]]
    except IndexError:
        # print("checking neighbours OOB", space)
        return neighbour_set
    
    # print("checking neighbours haha", space, current_value) 
    if space in neighbour_set or current_value != ".":
        return neighbour_set
    else:
        neighbour_set.add(space)
        if space[0] == 0:
            # in hallway
            if space[1] == 0:
                # only neighbour is to the right
                neighbour_set = check_neighbours((0, 1), field, neighbour_set)
            elif space[1] == field.shape[1] - 1:
                # only neighbour is to the left
                neighbour_set = check_neighbours((0, space[1] - 1), field, neighbour_set)
            else:
                neighbour_set = check_neighbours((space[0], space[1]-1), field, neighbour_set)
                neighbour_set = check_neighbours((space[0], space[1]+1), field, neighbour_set)
                neighbour_set = check_neighbours((space[0]+1, space[1]), field, neighbour_set)
        else:
            # in burrow - can only go up or down
            if space[0] < field.shape[0] - 1:
                neighbour_set = check_neighbours((space[0]-1, space[1]), field, neighbour_set)
                neighbour_set = check_neighbours((space[0]+1, space[1]), field, neighbour_set)
            else:
                # print("this space is in a burrow haha")
                # in bottom of burrow - can only go up
                neighbour_set = check_neighbours((space[0]-1, space[1]), field, neighbour_set)
        return neighbour_set

In [48]:
# Define a class for amphipods to use
class Amphipod:
    burrow_entrances = {(0, 2), (0, 4), (0, 6), (0, 8)}
    num_to_species = {1: "A", 2: "B", 3: "C", 4: "D"}
    species_to_burrow = {"A": 2, "B": 4, "C": 6, "D": 8}
    energy_cost_dict = {"A": 1, "B": 10, "C": 100, "D": 1000}
    
    __slots__ = ("y", "x", "species", "energy_cost", "endgame")
    
    def __init__(self, position, species):
        self.y = position[0]
        self.x = position[1]
        self.species = species  # either "A", "B", "C", or "D"
        self.energy_cost = self.energy_cost_dict[species]
        self.endgame = False
        
        
    def __deepcopy__(self, memo):
        copy_amp = Amphipod((self.y, self.x), self.species)
        memo[id(self)] = copy_amp
        return copy_amp

        
    def gen_moves(self, field):
        # return a set of all spaces it can move to
        neighbour_set = set()
        
        if self.is_endgame(field):  # if it shouldn't move at all
            return neighbour_set
        # print("this field:")
        # print(field)

        for i in (-1, +1):
            # print(f"checking neighbours for {(self.y+i, self.x)}")
            new_neighbour_set = check_neighbours((self.y+i, self.x), field, neighbour_set)
            # print("new moves:", new_neighbour_set)
            neighbour_set = neighbour_set | new_neighbour_set
            # print("current moves:", neighbour_set)
            
        for i in (-1, +1):
            # print(f"checking neighbours for {(self.y, self.x+j)}")
            new_neighbour_set = check_neighbours((self.y, self.x+i), field, neighbour_set)
            neighbour_set = neighbour_set | new_neighbour_set
            # print("current moves:", neighbour_set)
            
        # print("all empty spaces:", neighbour_set)
        
        # "clean" the possible moves
        # can't stay in same place
        neighbour_set.discard((self.y, self.x))
        
        # can't stop in front of burrow - could use difference of sets!
        neighbour_set -= self.burrow_entrances

        # print("no burrow entrances:", neighbour_set)
        
        # if in hallway, can't move except to its own burrow - try using set filtering stuff
        burrow_column = self.species_to_burrow[self.species]
        if self.y == 0:                
            neighbour_set = {space for space in neighbour_set if not (space[0] == 0 or space[1] != burrow_column)}
        # print("in hallway:", neighbour_set)
                
        # can't move to any burrow that's not its own ever
        neighbour_set = {space for space in neighbour_set if not (space[0] > 0 and space[1] != burrow_column)}
        # print("no unowned burrow:", neighbour_set)
            
        # can only move to its own burrow if it's empty or has only its own species
        own_burrow = field[1:, burrow_column]
        
        # # need some logic here - if there is anything but . and species here
        if np.all((own_burrow == ".") | (own_burrow == self.species)):
            pass
        else: 
            neighbour_set = {space for space in neighbour_set if not (space[0] > 0 and space[1] == burrow_column)}
        # print("no own burrow in some cases:", neighbour_set)
                
        # if it can still move to its own column, then do that
        for i in range(field.shape[0] - 1, 0, -1):
            if (i, burrow_column) in neighbour_set and field[i, burrow_column] == ".":
                neighbour_set = {(i, burrow_column)}
                break
        # print("can move to own column?:", neighbour_set)
            
        # that's all the logic I think?
        return neighbour_set
    
    
    def moveto(self, position):
        "Changes position of amphipod. Returns cost of moving to that space."
        up_steps = self.y
        across_steps = abs(self.x - position[1])
        down_steps = position[0]

        energy_cost = (up_steps + across_steps + down_steps) * self.energy_cost
        
        self.y = position[0]
        self.x = position[1]
        return energy_cost
    
    
    def is_endgame(self, field):
        """Returns True if this amphipod shouldn't move again, False otherwise.
        Sets an attribute to get once is_endgame is True."""
        if self.endgame:
            return True
        else:
            burrow_column = self.species_to_burrow[self.species]
            if self.x == burrow_column:  # is at bottom of its burrow
                if self.y == field.shape[0] - 1:
                    return True
                # elif field[2, burrow_column] == self.species and self.y == 1:  # or at top of burrow but same species is at bottom
                #     return True
                elif np.all(field[self.y:, self.x] == self.species) and self.y > 0:
                    self.endgame = True
                    return True
            else:
                return False

In [5]:
def get_burrow(field, species):
    "Returns an array of the burrow for species showing its state. Left is top, right is bottom."
    return {"A": field[1:, 2], "B": field[1:, 4], "C": field[1:, 6], "D": field[1:, 8]}[species]

In [6]:
def get_estimate_to_win(field, amphipods):
    # Make estimate on cost to win from this point - must be least possible estimate
    # if larger than the current best guess, stop doing thigns - won't be able to win from this point
    estimate_to_win = 0
    not_endgame = []
    yes_endgame = []
    for amp in amphipods:
        yes_endgame.append(amp) if amp.is_endgame(field) else not_endgame.append(amp)
    
    # Have two lists of endgame and non-endgame amphipods
    # Need list of spaces each could move to
    not_full = copy.deepcopy(ENDGAME_POSITIONS)
    for amp in yes_endgame:
        # print("endgame amp position:", amp.species, (amp.y, amp.x))
        # print(not_full[amp.species])
        not_full[amp.species].remove((amp.y, amp.x))
        
    amp_target = []  # list of tuples; (amphipod, target position)
    for amp in not_endgame:
        target_pos = not_full[amp.species].pop()
        amp_target.append((amp, target_pos))
        
        up_steps = amp.y
        across_steps = abs(amp.x - target_pos[1])
        down_steps = target_pos[0]
        estimate_to_win += (up_steps + across_steps + down_steps) * amp.energy_cost
        
    return estimate_to_win

In [23]:
SPECIES_TO_BURROW = {"A": 2, "B": 4, "C": 6, "D": 8}

def get_cost(target_pos, amp):
    up_steps = amp.y
    across_steps = abs(amp.x - target_pos[1])
    down_steps = target_pos[0]
    return (up_steps + across_steps + down_steps) * amp.energy_cost
    

def play_move(field, amphipods, amp_to_move, move, best_guess, energy_cost, game_state_cost, moves):
    "Returns best_guess and game_state_cost for future stuff :/"
    # Make copies of things so recursion works fine
    field = field.copy()
    amphipods = copy.deepcopy(amphipods)
    # moves = copy.deepcopy(moves)
    
    # Make move
    if move is not None:
        this_amp = amphipods[amp_to_move]
        field[this_amp.y, this_amp.x] = "."  # old position is now empty
        energy_to_move = this_amp.moveto(move)
        energy_cost += energy_to_move
        field[this_amp.y, this_amp.x] = this_amp.species  # new position is now full
        # moves.append((field, energy_cost))
        
    # Check if need to stop early    
    if energy_cost >= best_guess:
        # Have already spent more energy than allowed, so this can't be a good way of doing it
        return best_guess, game_state_cost
        
    estimate_to_win = get_estimate_to_win(field, amphipods)
    if energy_cost + estimate_to_win >= best_guess:
        return best_guess, game_state_cost
    
    # Give each non-endgame amphipod a target location, then work out the energy to move to that target location
    # sum up these energies - this is the good estimate to use
    
    # check if game is over
    if check_won(field, amphipods):
        print("Game is over!!! energy_cost", energy_cost, "best_guess", best_guess)
        
        # if energy_cost < 12521:
        #     print("all moves:")
        #     for i in moves:
        #         print(i[0])  # the field
        #         print("cost so far:", i[1])  # the cost
        #     raise Exception("Stop please")
            
        if energy_cost < best_guess:
            best_guess = energy_cost
            
        return best_guess, game_state_cost
    
    # State is concrete now, so hash it and save cost
    field_bytes = field.tobytes()
    if field_bytes in game_state_cost and game_state_cost[field_bytes] <= energy_cost:
        # print("have reached state I've reached before hmmm")
        return best_guess, game_state_cost  # have already reached this state with a better cost already
    else:
        game_state_cost[field_bytes] = energy_cost
        
    # this game state still could be worthwhile exploring
    # so play every possible move from here
    # then inspect those moves
    for amp_to_move, amp in enumerate(amphipods):  # order by cost of movement
        possible_moves = amp.gen_moves(field)  # all possible positions this amphipod could move to
        for move in sorted(possible_moves, key=lambda x: get_cost(x, amp), reverse=True):  # sort by descending cost of move
            # Inspect that move
            best_guess, game_state_cost = play_move(field, amphipods, amp_to_move, move, best_guess, energy_cost, game_state_cost, moves)
            
    return best_guess, game_state_cost

In [49]:
import collections
import numpy as np
import copy
import time

num_to_species = {1: "A", 2: "B", 3: "C", 4: "D"}

lines = []
# with open("ex1.txt") as f:
with open("amphipods.txt") as f:
    hallway_index = 1000
    burrows = collections.defaultdict(list)
    burrow_connection = {}
    for index, line in enumerate(f.readlines()):
        # create field
        line_list = [char for char in line.strip()]
        if lines:
            while len(line_list) < len(lines[0]):
                line_list = ["#"] + line_list + ["#"]
        lines.append(line_list)

        # create other stuff
        line = line.strip()
        if "." in line:
            hallway_index = index
            hallway = ["."] * line.count(".")
        if index > hallway_index:
            n_burrow = 1
            for char_index, char in enumerate(line):
                if char != "#":
                    if n_burrow not in burrow_connection:
                        burrow_connection[num_to_species[n_burrow]] = char_index - 1
                    burrows[num_to_species[n_burrow]].append(char)
                    n_burrow += 1
                    
field = np.array(lines)
field = field[1:-1, 1:-1]
print(field)

burrow_height = field.shape[0] - 1

ENDGAME_POSITIONS = {}
for species, column in (("A", 2), ("B", 4), ("C", 6), ("D", 8)):
    print(species, column)
    ENDGAME_POSITIONS[species] = []
    for row in range(burrow_height):
        ENDGAME_POSITIONS[species].append((row+1, column))

for i in ENDGAME_POSITIONS.items():
    print(i)

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['#' '#' 'D' '#' 'C' '#' 'B' '#' 'C' '#' '#']
 ['#' '#' 'D' '#' 'A' '#' 'A' '#' 'B' '#' '#']]
A 2
B 4
C 6
D 8
('A', [(1, 2), (2, 2)])
('B', [(1, 4), (2, 4)])
('C', [(1, 6), (2, 6)])
('D', [(1, 8), (2, 8)])


In [53]:
amphipods = []
for index, element in np.ndenumerate(field):
    if element in ("ABCD"):
        amphipods.append(Amphipod(index, element))
        
# sort amphipods by cost to move
amphipods.sort(key=lambda x: x.energy_cost, reverse=True)
        
best_guess = 19115  # worked out by hand, AoC says it's too high
energy_cost = 0
game_state_cost = {}
moves = [(field, 0)]

# Should get 12521

start = time.time()
best_guess, game_state_cost = play_move(field, amphipods, 0, None, best_guess, energy_cost, game_state_cost, moves)
print("time taken was:", time.time() - start)
print(best_guess)

Game is over!!! energy_cost 19111 best_guess 19115
Game is over!!! energy_cost 19071 best_guess 19111
Game is over!!! energy_cost 19059 best_guess 19071
time taken was: 0.7799975872039795
19059


In [34]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
  %lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These stat

In [54]:
amphipods = []
for index, element in np.ndenumerate(field):
    if element in ("ABCD"):
        amphipods.append(Amphipod(index, element))
        
# sort amphipods by cost to move
amphipods.sort(key=lambda x: x.energy_cost, reverse=True)
        
best_guess = 19115  # worked out by hand, AoC says it's too high
energy_cost = 0
game_state_cost = {}
moves = [(field, 0)]

# Should get 12521

%lprun -f play_move play_move(field, amphipods, 0, None, best_guess, energy_cost, game_state_cost, moves)

# start = time.time()
# best_guess, game_state_cost = play_move(field, amphipods, 0, None, best_guess, energy_cost, game_state_cost, moves)
# print("time taken was:", time.time() - start)
# print(best_guess)

Game is over!!! energy_cost 19111 best_guess 19115
Game is over!!! energy_cost 19071 best_guess 19111
Game is over!!! energy_cost 19059 best_guess 19071


Timer unit: 1e-07 s

Total time: 2.55549 s
File: C:\Users\Gwion\AppData\Local\Temp/ipykernel_12576/459589137.py
Function: play_move at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def play_move(field, amphipods, amp_to_move, move, best_guess, energy_cost, game_state_cost, moves):
    11                                               "Returns best_guess and game_state_cost for future stuff :/"
    12                                               # Make copies of things so recursion works fine
    13      2220     168159.0     75.7      0.7      field = field.copy()
    14      2220    4151681.0   1870.1     16.2      amphipods = copy.deepcopy(amphipods)
    15                                               # moves = copy.deepcopy(moves)
    16                                               
    17                                               # Make move
    18      2220      45283.0     20.4      0.2      if move 

## Part 2 (for later)

In [42]:
import collections
import numpy as np
import copy
import time

num_to_species = {1: "A", 2: "B", 3: "C", 4: "D"}

lines = []
# with open("ex1.txt") as f:
with open("amphipods2.txt") as f:
    hallway_index = 1000
    burrows = collections.defaultdict(list)
    burrow_connection = {}
    for index, line in enumerate(f.readlines()):
        # create field
        line_list = [char for char in line.strip()]
        if lines:
            while len(line_list) < len(lines[0]):
                line_list = ["#"] + line_list + ["#"]
        lines.append(line_list)

        # create other stuff
        line = line.strip()
        if "." in line:
            hallway_index = index
            hallway = ["."] * line.count(".")
        if index > hallway_index:
            n_burrow = 1
            for char_index, char in enumerate(line):
                if char != "#":
                    if n_burrow not in burrow_connection:
                        burrow_connection[num_to_species[n_burrow]] = char_index - 1
                    burrows[num_to_species[n_burrow]].append(char)
                    n_burrow += 1
                    
field = np.array(lines)
field = field[1:-1, 1:-1]
print(field)

burrow_height = field.shape[0] - 1

ENDGAME_POSITIONS = {}
for species, column in (("A", 2), ("B", 4), ("C", 6), ("D", 8)):
    print(species, column)
    ENDGAME_POSITIONS[species] = []
    for row in range(burrow_height):
        ENDGAME_POSITIONS[species].append((row+1, column))

for i in ENDGAME_POSITIONS.items():
    print(i)

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['#' '#' 'D' '#' 'C' '#' 'B' '#' 'C' '#' '#']
 ['#' '#' 'D' '#' 'C' '#' 'B' '#' 'A' '#' '#']
 ['#' '#' 'D' '#' 'B' '#' 'A' '#' 'C' '#' '#']
 ['#' '#' 'D' '#' 'A' '#' 'A' '#' 'B' '#' '#']]
A 2
B 4
C 6
D 8
('A', [(1, 2), (2, 2), (3, 2), (4, 2)])
('B', [(1, 4), (2, 4), (3, 4), (4, 4)])
('C', [(1, 6), (2, 6), (3, 6), (4, 6)])
('D', [(1, 8), (2, 8), (3, 8), (4, 8)])


In [47]:
old_field = field.copy()

start = time.time()
# Should get 48541

field = old_field.copy()

amphipods = []
for index, element in np.ndenumerate(field):
    if element in ("ABCD"):
        amphipods.append(Amphipod(index, element))
# sort amphipods by cost to move
amphipods.sort(key=lambda x: x.energy_cost, reverse=True)

best_guess = 65536
energy_cost = 0
game_state_cost = {}
moves = [(field, 0)]
print(f"Using initial guess of {best_guess}...")
%lprun -f play_move play_move(field, amphipods, 0, None, best_guess, energy_cost, game_state_cost, moves)
# result, game_state_cost = play_move(field, amphipods, 0, None, best_guess, energy_cost, game_state_cost, moves)
# print(f"Result was {result}")
    
# print("time taken was:", time.time() - start)
# print(result)

Using initial guess of 65536...
Game is over!!! energy_cost 49263 best_guess 65536
Game is over!!! energy_cost 49261 best_guess 49263
Game is over!!! energy_cost 48543 best_guess 49261
Game is over!!! energy_cost 48541 best_guess 48543


Timer unit: 1e-07 s

Total time: 532.403 s
File: C:\Users\Gwion\AppData\Local\Temp/ipykernel_12576/459589137.py
Function: play_move at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def play_move(field, amphipods, amp_to_move, move, best_guess, energy_cost, game_state_cost, moves):
    11                                               "Returns best_guess and game_state_cost for future stuff :/"
    12                                               # Make copies of things so recursion works fine
    13    189497   14403679.0     76.0      0.3      field = field.copy()
    14    189497  705569018.0   3723.4     13.3      amphipods = copy.deepcopy(amphipods)
    15                                               # moves = copy.deepcopy(moves)
    16                                               
    17                                               # Make move
    18    189497    4997794.0     26.4      0.1      if move 